# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jacareacanga,-6.2222,-57.7528,22.25,99,100,0.62,BR,1675668360
1,1,upernavik,72.7868,-56.1549,-26.60,94,100,7.03,GL,1675668157
2,2,kargat,55.1956,80.2811,-11.63,91,100,2.87,RU,1675668361
3,3,warrington,53.3925,-2.5802,1.69,87,0,2.06,GB,1675668361
4,4,hithadhoo,-0.6000,73.0833,27.48,71,25,5.71,MV,1675668082


In [5]:
city_data_df.count()

City_ID       553
City          553
Lat           553
Lng           553
Max Temp      553
Humidity      553
Cloudiness    553
Wind Speed    553
Country       550
Date          553
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
#%%capture --no-display

# Configure the map plot
humdity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles= "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    scale = 1.0
    ) 

# Display the map
humdity_map 


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
your_ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] >=20) & (city_data_df["Max Temp"]<=25) & (city_data_df["Wind Speed"]<=4.00) & (city_data_df["Cloudiness"]<=50) & (city_data_df["Humidity"]<=55), :]
# Drop any rows with null values
# YOUR CODE HERE
ideal_weather = your_ideal_weather.dropna(how = "any")
# Display sample data
# YOUR CODE HERE
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
123,123,tura,25.5198,90.2201,24.94,29,0,2.75,IN,1675668396
297,297,tikapur,28.5000,81.1333,24.42,33,0,2.80,NP,1675668454
319,319,chipinge,-20.1883,32.6236,24.27,48,0,1.64,ZW,1675668460
327,327,kemri,28.8167,79.2167,23.46,31,0,3.48,IN,1675668462
329,329,umm lajj,25.0213,37.2685,20.48,53,9,1.10,SA,1675668463


In [8]:
ideal_weather.count()

City_ID       8
City          8
Lat           8
Lng           8
Max Temp      8
Humidity      8
Cloudiness    8
Wind Speed    8
Country       8
Date          8
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df= ideal_weather.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"]= ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
123,123,tura,25.5198,90.2201,24.94,29,0,2.75,IN,1675668396,
297,297,tikapur,28.5000,81.1333,24.42,33,0,2.80,NP,1675668454,
319,319,chipinge,-20.1883,32.6236,24.27,48,0,1.64,ZW,1675668460,
327,327,kemri,28.8167,79.2167,23.46,31,0,3.48,IN,1675668462,
329,329,umm lajj,25.0213,37.2685,20.48,53,9,1.10,SA,1675668463,
352,352,bahir dar,11.5936,37.3908,24.01,22,22,1.30,ET,1675668469,
467,467,san bernardo,-33.6000,-70.7167,23.41,50,0,0.71,CL,1675668498,
470,470,kedougou,12.5579,-12.1743,22.32,35,0,0.00,SN,1675668499,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel

radius = 10000 # YOUR CODE HERE
limit = 20
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": 20
}
    
 
   
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get city's name
    city = row["City"]
    
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    Latitude = row["Lat"]
    Longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{Longitude},{Latitude}" # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = response.json() # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    #print(f"nearest hotel from {city}: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: No hotel found
tikapur - nearest hotel: Banana Agro Resort
chipinge - nearest hotel: No hotel found
kemri - nearest hotel: No hotel found
umm lajj - nearest hotel: No hotel found
bahir dar - nearest hotel: ናኪ ሆቴል
san bernardo - nearest hotel: No hotel found
kedougou - nearest hotel: Campement Chez Diao


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
123,123,tura,25.5198,90.2201,24.94,29,0,2.75,IN,1675668396,No hotel found
297,297,tikapur,28.5000,81.1333,24.42,33,0,2.80,NP,1675668454,Banana Agro Resort
319,319,chipinge,-20.1883,32.6236,24.27,48,0,1.64,ZW,1675668460,No hotel found
327,327,kemri,28.8167,79.2167,23.46,31,0,3.48,IN,1675668462,No hotel found
329,329,umm lajj,25.0213,37.2685,20.48,53,9,1.10,SA,1675668463,No hotel found
352,352,bahir dar,11.5936,37.3908,24.01,22,22,1.30,ET,1675668469,ናኪ ሆቴል
467,467,san bernardo,-33.6000,-70.7167,23.41,50,0,0.71,CL,1675668498,No hotel found
470,470,kedougou,12.5579,-12.1743,22.32,35,0,0.00,SN,1675668499,Campement Chez Diao


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles= "EsriNatGeo",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    hover_cols= ["Hotel Name", "Country"],
    scale = 4.0
    ) 

# Display the map
hotel_map 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)